代理模式是为一个对象提供一个代用品或占位符，以便控制对它的访问。

In [2]:
(function(){
    var Flower = function(){}
    
    var xiaoming = {
        sendFlower:function(target){
            var flower = new Flower();
            target.receiveFlower(flower);
        }
    }
    
    var A = {
        receiveFlower:function(flower){
            console.log('收到花', flower);
        }
    }
    
    xiaoming.sendFlower(A)
    
})()

收到花 Flower {}


In [4]:
(function(){
    var Flower= function(){}
    
    var xiaoming = {
        sendFlower:function(target){
            var flower = new Flower();
            target.receiveFlower(flower);
        }
    }
    
    var B = {
        receiveFlower:function(flower){
            A.receiveFlower(flower)
        }
    }
    
    var A = {
        receiveFlower:function(flower){
            console.log('收到花',flower);
        }
    }
    
    xiaoming.sendFlower(B)
})()

收到花 Flower {}


In [6]:
(function(){
    var Flower= function(){}
    
    var xiaoming = {
        sendFlower:function(target){
            var flower = new Flower();
            target.receiveFlower(flower);
        }
    }
    
    var B = {
        receiveFlower:function(flower){
            A.listenGoodMood(function(){
                A.receiveFlower(flower)
            })            
        }
    }
    
    var A = {
        receiveFlower:function(flower){
            console.log('收到花',flower);
        },
        listenGoodMood:function(fn){
            setTimeout(function(){
                fn()
            },1000);
        }
    }
    
    xiaoming.sendFlower(B)
})()

收到花 Flower {}


代理模式

- 虚拟代理 开销很大的对象，延迟到真正需要它的时候再去创建
- 保护代理 控制对象访问

In [ ]:
(function(){
    var myImage = (function(){
        var imgNode = document.createElement('img');
        document.body.appendChild(imgNode);
        
        return {
            setSrc:function(src){
                imgNode.src = src
            }
        }
    })()
    
    var proxyImage = (function(){
        var img = new Image;
        img.onload = function(){
            myImage.setSrc(this.src)
        }
        
        return {
            setSrc:function(src){
                myImage.setSrc('local.gif')
                img.src = src;
            }
        }
    })()
    
    proxyImage.setSrc('new.gif')
})()

单一职责原则：一个类（通常也包括对象和函数等）而言，应该仅有一个引起它变化的原因。

面向接口编程迎合依赖倒置原则，通过接口向上转型，从而避开了编译器的类型检查。

In [ ]:
(function(){
    var synchronousFile = function(id){
        console.log('start sync, id is',id);
    }
    
    var proxySynchronousFile = (function(){
        var cache = [],
            timer;
        
        return function(id){
            cache.push(id);
            if(timer){
                return
            }
            
            timer = setTimeout(function(){
                synchronousFile(cache.join(','));
                clearTimeout(timer);
                timer = null;
                cache.length = 0;
            }, 2000);
        }
    })
})()

In [ ]:
(function(){
    var miniConsole = (function(){
        var cache = [];
        var handler = function(ev){
            if(ev.keyCode === 113){
                var script = document.createElement('script');
                script.onload = function(){
                    for(var i =0,fn;fn = cache[i++];){
                        fn()
                    }
                }
                
                script.src = 'miniConsole.js';
                document.getElementsByTagName('head')[0].appendChild(script);
                document.body.removeEventListener('keydown',handler);
            }
        }
        document.body.addEventListener('keydown',handler,false);
        
        return {
            log:function(){
                var args = arguments;
                cache.push(function(){
                    return miniConsole.log.apply(miniConsole,args);
                })
            }
        }
    })()
    
    miniConsole.log(11);
})()

In [ ]:
//缓存代理
(function(){
    var mult = function(){
        console.log('开始计算');
        var a= 1;
        for(var i = 0,l= arguments.length;i<l;i++){
            a = a* arguments[i];
        }
        
        return a;
    }
    
    var proxyMult = (function(){
        var cache = {};
        return function(){
            var args = Array.prototype.join.call(arguments,',');
            if(args in cache){
                return cache[args];
            }
            return cache[args] = mult.apply(this, arguments);
        }
    })();
    
    proxyMult(1,2,3)
    proxyMult(1,2,3)
})()

In [ ]:
(function(){
    var mult = function(){
        var a = 1;
        for(var i =0,l = arguments.length;i<l;i++){
            a = a* arguments[i]
        }
        return a;
    }
    
    var plus = function(){
        var a = 0;
        for(var i =0,l = arguments.length;i<l;i++){
            a = a+arguments[i]
        }
        return a;
    }
    
    var createProxyFactory = function(fn){
        var cache = {};
        return function(){
            var args = Array.prototype.join.call(arguments,',');
            if(args in cache){
                return cache[args];
            }
            return cache[args] = fn.apply(this, arguments);
        }
    }
    
    var proxyMult = createProxyFactory(mult),
        proxyPlus = createProxyFactory(plus);
})()